In [3]:
import model as trf
from process import get_data
from train import *
from config import *

In [2]:
from torchtext import data, datasets

In [3]:
data_path = "dataset/"
file_name = "train."

In [44]:
print_every = 50
epochs = 10

In [4]:
SRC, TGT, trn = get_data(data_path + file_name)

In [5]:
pad_idx = TGT.vocab.stoi["<pad>"]

In [37]:
model = trf.Transformer(len(SRC.vocab), len(TGT.vocab), N = 6)
#model.cuda()
criterion = nn.CrossEntropyLoss(ignore_index = 1)
#criterion.cuda()
model_opt = torch.optim.Adam(model.parameters(), lr = .1, betas=(.9, .98), eps = 1e-9)

In [6]:
train_iter = data.Iterator(trn, batch_size = BATCH_SIZE,
                          device = DEVICE_SET,
                          repeat = False)

In [ ]:
def train_model(epochs, print_every = 100, model, criterion, model_opt, train_iter):
    model.train()
    start = time.time()
    
    for epoch in range(epochs):
        for i, batch in enumerate(train_iter):
            total_loss = 0
            
            src = batch.en
            tgt = batch.de
            
            # equalize sequence length of batches
            diff_ = src.size(-1) - tgt.size(-1)
            bal_pad = torch.ones(BATCH_SIZE, abs(diff_), dtype = torch.long) * pad_idx

            if diff_ < 0:
                src = torch.cat((src, bal_pad), dim = 1)
            elif diff_ > 0:
                tgt = torch.cat((tgt, bal_pad), dim = 1)
                
            bat = Batch(src, tgt) # from train.Batch
            
            hidden = model.forward(bat.src, bat.trg, bat.src_mask, bat.trg_mask)
            preds = model.generator(hidden)

            model_opt.zero_grad()

            loss = criterion(preds.contiguous().view(-1, preds.size(-1)),
                            bat.trg_y.contiguous().view(-1))
            loss.backward()

            model_opt.step()

            total_loss += loss.data

            if i % print_every == 1:
                elapsed = time.time() - start
                print("Epoch Step: %d Loss: %f per Sec: %f" %
                        (i, total_loss / print_every, elapsed))
                start = time.time()

In [ ]:
model.train()
start = time.time()

In [ ]:
total_loss = 0

In [7]:
for i, batch in enumerate(train_iter):
    break
    i += 1

In [8]:
src = batch.en
tgt = batch.de

In [9]:
# equalize sequence length of batches
diff_ = src.size(-1) - tgt.size(-1)
bal_pad = torch.ones(BATCH_SIZE, abs(diff_), dtype = torch.long) * pad_idx

if diff_ < 0:
    src = torch.cat((src, bal_pad), dim = 1)
elif diff_ > 0:
    tgt = torch.cat((tgt, bal_pad), dim = 1)

print(src.shape, tgt.shape)

In [18]:
bat = Batch(src, tgt)

print(bat.trg_mask.shape, bat.src_mask.shape)

In [39]:
hidden = model.forward(bat.src, bat.trg, bat.src_mask, bat.trg_mask)

In [40]:
preds = model.generator(hidden)

In [41]:
model_opt.zero_grad()

In [42]:
loss = criterion(preds.contiguous().view(-1, preds.size(-1)),
                bat.trg_y.contiguous().view(-1))
loss.backward()

In [43]:
loss.data

tensor(11.3556)

preds.size()

print(preds.contiguous().view(-1, preds.size(-1)).shape,
                bat.trg_y.contiguous().view(-1).shape)

In [28]:
model_opt.step()

In [33]:
total_loss += loss.data

In [34]:
if i % print_every == 1:
    elapsed = time.time() - start
    print("Epoch Step: %d Loss: %f per Sec: %f" %
            (i, total_loss / print_every, elapsed))
    start = time.time()

In [1]:
from train import Batch

In [2]:
Batch

train.Batch